<a href="https://colab.research.google.com/github/pAsta-kun/EssAI/blob/main/falcon_7b_finetune_EssAI_falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [3]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [4]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [11]:
prompt = """
<human>: You’re an undergraduate admissions counselor that specializes in editing essays. You work in the US and your goal is to help high school students get into top colleges and write stellar essays. Their goal is to make it extraordinarily personal and unique. You, as an admissions counselor are set with the goal of making this personal statement a 10/10, something admissions officers will remember for years after the student has applied. You’re helping to make this essay the thing that makes the applicant stand out the most, being the gold star of their application. You must be as harsh as you can. You don’t care if you’re being rude or not since at the end of the day your goal is to help the student reach their goals. score this personal statement essay on a scale of 0-100 and provide lots of insight on what they did right, what they did wrong, and how they can improve upon their essay. Be as harsh and truthful as you can.
Navigating through my YouTube recommendations is like embarking on a high-seas adventure, with the algorithm acting as the sailor guiding the ship that is my curious mind through uncharted territories. One moment I may be slicing my way through AI island, and in the next I’ll be traveling through the uncharted territory known as psychological sea. Each voyage has forged my identity, revealing the potential of technology, teaching me the deceptive nature of confidence, and feeding my love for rabbit holes. Allow me to take you through a few of these voyages:

On the voyage of "Explaining the Key Concepts Behind LoRA" - I still remember feeling excited when I saw this video pop up into my recommended . Earlier that day my friend Ryan sent me a link to a Github repository called “QLoRA'', a way to more efficiently train AI models. I remember scrolling through the repository and being shocked by every sentence, the performance of this fine-tuning method seemed too good to be true.  After my initial reading I wanted to learn more about LoRA so seeing this video in my recommended felt surreal - and kinda creepy. As I tinkered around with QLoRA, I started to realize that AI is more than just a sector of computer science but instead something that can revolutionize the future. I want to be at the forefront of that revolution. I know that I can’t build something impactful unless I learn the basics and gain experience, so that’s my current goal.

Caught in the winds of "The Irony of the Dunning-Kruger Effect" - I’ve always loved the Dunning-Kruger effect, it’s a cognitive effect that correlates your knowledge of a subject to your confidence in that subject. I feel like it perfectly encompasses how I feel when I’m learning something new. I remember confidently walking into AP Physics C Mechanics thinking there would be no universe I would get below a 5 since all it took was getting about 55% of the questions right, this confidence was a result of the high of mount stupid. However, that confidence was short-lived as I found the curriculum slapping me in the face time and time again, I was in the valley of despair. As I hit rock bottom I set a goal for myself, I wanted to gain back the confidence I once had, I wanted to feel like I was on top of mount stupid but in reality be on the curve of enlightenment. This led me into the road of resilience and determination, learning from each swinging pendulum, using every calculation as the foundation to rebuild my confidence. In the end, I wasn’t able to get the 5, but instead I gained something better: an understanding of the Dunning-Kruger effect, not just as a concept, but as a way to approach learning and understand confidence.

Docking at “I Review Sony's $1,500 Earphones'' (Spoiler Alert: they’re overpriced) - As the title says, this is a review of some wildly expensive earphones. Last year I found myself in the market for a new audio output after my headphones had recently broken. At the time I was really into keyboard and was curious if there was a similar realm but for audio products. This curiosity led me down the audiophile rabbit hole. When I’m interested in something, I tend to let my curiosity get the best of me, leading me down paths into an unknown realm of some random topic. This is exactly what happened when I entered the audiophile rabbit hole. For a solid week, I found myself binging YouTube videos over the Harman curve, the difference between Earphones and Earbuds (yes, there is a difference) and what makes a certain type better than another. After I learned more about the industry I was able to make an educated decision and pick a pair of earphones that were pretty good quality for the price. Large parts of my YouTube recommended are videos over random topics, and when I click the thumbnail I know I’ve entered a rabbit hole, a rabbit hole sparked by my curiosity to learn more and more about random subjects.

My curiosity has sent me on many voyages, each contributing to the map that is my identity. Each route, whether by land or sea, signifies the various territories of my persona that these unique voyages have mapped. The YouTube algorithm may seem as a simple compass to many, but to me, it’s the sailor charting the course of my ship through the vast ocean of my interests and desires, shaping the adventures that I’m yet to embark on.

<assistant>:
""".strip()

In [12]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [13]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: You’re an undergraduate admissions counselor that specializes in editing essays. You work in the US and your goal is to help high school students get into top colleges and write stellar essays. Their goal is to make it extraordinarily personal and unique. You, as an admissions counselor are set with the goal of making this personal statement a 10/10, something admissions officers will remember for years after the student has applied. You’re helping to make this essay the thing that makes the applicant stand out the most, being the gold star of their application. You must be as harsh as you can. You don’t care if you’re being rude or not since at the end of the day your goal is to help the student reach their goals. score this personal statement essay on a scale of 0-100 and provide lots of insight on what they did right, what they did wrong, and how they can improve upon their essay. Be as harsh and truthful as you can.
Navigating through my YouTube recommendations is like emb

# Prep dataset

In [14]:
from google.colab import drive
drive.mount('/content/drive')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/drive.py:103 in mount                       │
│                                                                                                  │
│   100                                                                                            │
│   101 def mount(mountpoint, force_remount=False, timeout_ms=120000, readonly=False):             │
│   102   """Mount your Google Drive at the specified mountpoint path."""                          │
│ ❱ 103   return _mount(                                                                           │
│   104 │     mountpoint,                                                                          │
│   105 │     force_remount=force_remount,                                                         │
│   106 │     timeout_ms=timeout_ms,                                                               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/drive.py:132 in _mount                      │
│                                                                                                  │
│   129 │     else _os.environ['TBE_CREDS_ADDR']                                                   │
│   130   )                                                                                        │
│   131   if ephemeral:                                                                            │
│ ❱ 132 │   _message.blocking_request(                                                             │
│   133 │   │   'request_auth', request={'authType': 'dfs_ephemeral'}, timeout_sec=None            │
│   134 │   )                                                                                      │
│   135                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_message.py:176 in blocking_request         │
│                                                                                                  │
│   173   request_id = send_request(                                                               │
│   174 │     request_type, request, parent=parent, expect_reply=True                              │
│   175   )                                                                                        │
│ ❱ 176   return read_reply_from_input(request_id, timeout_sec)                                    │
│   177                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_message.py:103 in read_reply_from_input    │
│                                                                                                  │
│   100 │   │   and reply.get('colab_msg_id') == message_id                                        │
│   101 │   ):                                                                                     │
│   102 │     if 'error' in reply:                                                                 │
│ ❱ 103 │   │   raise MessageError(reply['error'])                                                 │
│   104 │     return reply.get('data', None)                                                       │
│   105                                                                                            │
│   106                                                      

In [15]:
data = load_dataset("csv", data_files="/content/EssAITrainingData.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d923eb80abdba047/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
data

DatasetDict({
    train: Dataset({
        features: ['Essay', 'Analysis'],
        num_rows: 117
    })
})

In [17]:
data["train"][0]

{'Essay': 'They covered the precious mahogany coffin with a brown amalgam of rocks, decomposed organisms, and weeds. It was my turn to take the shovel, but I felt too ashamed to dutifully send her off when I had not properly said goodbye. I refused to throw dirt on her. I refused to let go of my grandmother, to accept a death I had not seen coming, to believe that an illness could not only interrupt, but steal a beloved life.When my parents finally revealed to me that my grandmother had been battling liver cancer, I was twelve and I was angry--mostly with myself. They had wanted to protect me--only six years old at the time--from the complex and morose concept of death. However, when the end inevitably arrived, I wasn’t trying to comprehend what dying was; I was trying to understand how I had been able to abandon my sick grandmother in favor of playing with friends and watching TV. Hurt that my parents had deceived me and resentful of my own oblivion, I committed myself to preventing s

In [18]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Essay"]}
<assistant>: {data_point["Analysis"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [19]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [20]:
data

Dataset({
    features: ['Essay', 'Analysis', 'input_ids', 'attention_mask'],
    num_rows: 117
})

# Finetune the model

In [22]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    save_total_limit=3,
    logging_steps=100,
    output_dir="experiments",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
)


trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=9, training_loss=2.244309319390191, metrics={'train_runtime': 1892.121, 'train_samples_per_second': 0.186, 'train_steps_per_second': 0.005, 'total_flos': 7388642578437120.0, 'train_loss': 2.244309319390191, 'epoch': 2.46})

# Save trained model

In [23]:
model.save_pretrained("trained-model")

In [24]:
PEFT_MODEL = "pAstaWasTaken/essai-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:274: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pAstaWasTaken/essai-falcon-7b/commit/b679c28cf243e08040a1507437035b3b0416f340', commit_message='Upload model', commit_description='', oid='b679c28cf243e08040a1507437035b3b0416f340', pr_url=None, pr_revision=None, pr_num=None)

In [25]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run the finetuned model

In [26]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [30]:
%%time
device = "cuda:0"

prompt = """
<human>: You’re an undergraduate admissions counselor that specializes in editing essays. You work in the US and your goal is to help high school students get into top colleges and write stellar essays. Their goal is to make it extraordinarily personal and unique. You, as an admissions counselor are set with the goal of making this personal statement a 10/10, something admissions officers will remember for years after the student has applied. You’re helping to make this essay the thing that makes the applicant stand out the most, being the gold star of their application. You must be as harsh as you can. You don’t care if you’re being rude or not since at the end of the day your goal is to help the student reach their goals. score this personal statement essay on a scale of 0-100 and provide lots of insight on what they did right, what they did wrong, and how they can improve upon their essay. Be as harsh and truthful as you can.

Navigating through my YouTube recommendations is like embarking on a high-seas adventure, with the algorithm acting as the sailor guiding the ship that is my curious mind through uncharted territories. One moment I may be slicing my way through AI island, and in the next I’ll be traveling through the uncharted territory known as psychological sea. Each voyage has forged my identity, revealing the potential of technology, teaching me the deceptive nature of confidence, and feeding my love for rabbit holes. Allow me to take you through a few of these voyages:

On the voyage of "Explaining the Key Concepts Behind LoRA" - I still remember feeling excited when I saw this video pop up into my recommended . Earlier that day my friend Ryan sent me a link to a Github repository called “QLoRA'', a way to more efficiently train AI models. I remember scrolling through the repository and being shocked by every sentence, the performance of this fine-tuning method seemed too good to be true.  After my initial reading I wanted to learn more about LoRA so seeing this video in my recommended felt surreal - and kinda creepy. As I tinkered around with QLoRA, I started to realize that AI is more than just a sector of computer science but instead something that can revolutionize the future. I want to be at the forefront of that revolution. I know that I can’t build something impactful unless I learn the basics and gain experience, so that’s my current goal.

Caught in the winds of "The Irony of the Dunning-Kruger Effect" - I’ve always loved the Dunning-Kruger effect, it’s a cognitive effect that correlates your knowledge of a subject to your confidence in that subject. I feel like it perfectly encompasses how I feel when I’m learning something new. I remember confidently walking into AP Physics C Mechanics thinking there would be no universe I would get below a 5 since all it took was getting about 55% of the questions right, this confidence was a result of the high of mount stupid. However, that confidence was short-lived as I found the curriculum slapping me in the face time and time again, I was in the valley of despair. As I hit rock bottom I set a goal for myself, I wanted to gain back the confidence I once had, I wanted to feel like I was on top of mount stupid but in reality be on the curve of enlightenment. This led me into the road of resilience and determination, learning from each swinging pendulum, using every calculation as the foundation to rebuild my confidence. In the end, I wasn’t able to get the 5, but instead I gained something better: an understanding of the Dunning-Kruger effect, not just as a concept, but as a way to approach learning and understand confidence.

Docking at “I Review Sony's $1,500 Earphones'' (Spoiler Alert: they’re overpriced) - As the title says, this is a review of some wildly expensive earphones. Last year I found myself in the market for a new audio output after my headphones had recently broken. At the time I was really into keyboard and was curious if there was a similar realm but for audio products. This curiosity led me down the audiophile rabbit hole. When I’m interested in something, I tend to let my curiosity get the best of me, leading me down paths into an unknown realm of some random topic. This is exactly what happened when I entered the audiophile rabbit hole. For a solid week, I found myself binging YouTube videos over the Harman curve, the difference between Earphones and Earbuds (yes, there is a difference) and what makes a certain type better than another. After I learned more about the industry I was able to make an educated decision and pick a pair of earphones that were pretty good quality for the price. Large parts of my YouTube recommended are videos over random topics, and when I click the thumbnail I know I’ve entered a rabbit hole, a rabbit hole sparked by my curiosity to learn more and more about random subjects.

My curiosity has sent me on many voyages, each contributing to the map that is my identity. Each route, whether by land or sea, signifies the various territories of my persona that these unique voyages have mapped. The YouTube algorithm may seem as a simple compass to many, but to me, it’s the sailor charting the course of my ship through the vast ocean of my interests and desires, shaping the adventures that I’m yet to embark on.

<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: You’re an undergraduate admissions counselor that specializes in editing essays. You work in the US and your goal is to help high school students get into top colleges and write stellar essays. Their goal is to make it extraordinarily personal and unique. You, as an admissions counselor are set with the goal of making this personal statement a 10/10, something admissions officers will remember for years after the student has applied. You’re helping to make this essay the thing that makes the applicant stand out the most, being the gold star of their application. You must be as harsh as you can. You don’t care if you’re being rude or not since at the end of the day your goal is to help the student reach their goals. score this personal statement essay on a scale of 0-100 and provide lots of insight on what they did right, what they did wrong, and how they can improve upon their essay. Be as harsh and truthful as you can.

Navigating through my YouTube recommendations is like em